In [1]:
SAMPLE_TEXT = """
[({(<(())[]>[[{[]{<()<>>
[(()[<>])]({[<{<<[]>>(
{([(<{}[<>[]}>{[]{[(<()>
(((({<>}<{<{<>}{[]{[]{}
[[<[([]))<([[{}[[()]]]
[{[{({}]{}}([{[{{{}}([]
{<[[]]>}<{[{[{[]{()[[[]
[<(<(<(<{}))><([]([]()
<{([([[(<>()){}]>(<<{{
<{([{{}}[<[[[<>{}]]]>[]]
"""

In [2]:
def tokenize_line(line):
    return list(line)

def parse_text(raw_text):
    return [tokenize_line(l) for l in raw_text.split("\n") if l]

def read_input():
    with open("input.txt", "rt") as f:
        return f.read()

In [3]:
CHUNK_DELIMS = {
    '(': ')',
    '[': ']',
    '{': '}',
    '<': '>',
}

WEIGHTS = {
    ')': 3,
    ']': 57,
    '}': 1197,
    '>': 25137,
}

OPENING = set(CHUNK_DELIMS.keys())
CLOSING = set(CHUNK_DELIMS.values())

def locate_corruption(line):
    stack = []
    for char in line:
        if char in OPENING:
            stack.append(char)
        elif char in CLOSING:
            if not stack:
                # line is invalid, not corrupted
                return None, []
            else:
                open = stack.pop()
                if CHUNK_DELIMS[open] != char:
                    # print(f'{"".join(line)}: Expected {CHUNK_DELIMS[open]} got {char}')
                    return char, stack
    return None, stack

def score_corruptions(lines):
    return sum(WEIGHTS.get(locate_corruption(line)[0], 0) for line in lines)

In [4]:
score_corruptions(parse_text(SAMPLE_TEXT))

26397

In [5]:
score_corruptions(parse_text(read_input()))

339411

In [6]:
CLOSING_WEIGHTS = {
    ')': 1,
    ']': 2,
    '}': 3,
    '>': 4,
}

def complete_line(remaining_chars):
    result = [CHUNK_DELIMS[c] for c in remaining_chars]
    result.reverse()
    return result

def score_line_completions(lines):

    def calc_score(completion):
        sum = 0
        for c in completion:
            sum *=5
            sum += CLOSING_WEIGHTS[c]
        return sum

    scores = []
    for line in lines:
        char, remaining = locate_corruption(line)
        if char or not remaining:
            continue
        completion = complete_line(remaining)
        score = calc_score(completion)
        # print("".join(line), "".join(completion), score)
        scores.append(score)
    return scores

In [7]:
scores = score_line_completions(parse_text(SAMPLE_TEXT))
scores.sort()
scores[len(scores) // 2]

288957

In [8]:
scores = score_line_completions(parse_text(read_input()))
scores.sort()
scores[len(scores) // 2]

2289754624